In [131]:
import pandas as pd
import numpy as np
from datetime import datetime
from haversine import haversine
import math

In [132]:
df = pd.read_csv('Airbnb_Data.csv')
df = df[df['city'] == 'NYC']
print(df.columns)

Index(['id', 'log_price', 'property_type', 'room_type', 'amenities',
       'accommodates', 'bathrooms', 'bed_type', 'cancellation_policy',
       'cleaning_fee', 'city', 'description', 'first_review',
       'host_has_profile_pic', 'host_identity_verified', 'host_response_rate',
       'host_since', 'instant_bookable', 'last_review', 'latitude',
       'longitude', 'name', 'neighbourhood', 'number_of_reviews',
       'review_scores_rating', 'thumbnail_url', 'zipcode', 'bedrooms', 'beds'],
      dtype='object')


In [133]:
print(len(df))
print(df.head())

32349
          id  log_price property_type        room_type  \
0    6901257   5.010635     Apartment  Entire home/apt   
1    6304928   5.129899     Apartment  Entire home/apt   
2    7919400   4.976734     Apartment  Entire home/apt   
10   5578513   4.605170     Apartment     Private room   
18  17589436   4.882802     Apartment  Entire home/apt   

                                            amenities  accommodates  \
0   {"Wireless Internet","Air conditioning",Kitche...             3   
1   {"Wireless Internet","Air conditioning",Kitche...             7   
2   {TV,"Cable TV","Wireless Internet","Air condit...             5   
10  {Internet,"Wireless Internet","Air conditionin...             2   
18  {TV,"Cable TV",Internet,"Wireless Internet","A...             2   

    bathrooms  bed_type cancellation_policy  cleaning_fee  ...   latitude  \
0         1.0  Real Bed              strict          True  ...  40.696524   
1         1.0  Real Bed              strict          True  ...  

In [134]:
# One hot encoding
print(df['property_type'].unique())

['Apartment' 'House' 'Loft' 'Hostel' 'Condominium' 'Townhouse' 'Other'
 'Bed & Breakfast' 'Timeshare' 'Serviced apartment' 'Guesthouse'
 'Bungalow' 'Boutique hotel' 'Guest suite' 'Boat' 'Villa' 'Dorm'
 'Vacation home' 'In-law' 'Chalet' 'Yurt' 'Cabin' 'Tent' 'Earth House'
 'Castle']


In [135]:
# Should be scaled as such:
# 0 - Entire home/apt
# 1 - Private room
# 2 - Shared room
print(df['room_type'].unique())

['Entire home/apt' 'Private room' 'Shared room']


In [136]:
# Should be one-hot-encoded:
print(df['amenities'].unique())

['{"Wireless Internet","Air conditioning",Kitchen,Heating,"Family/kid friendly",Essentials,"Hair dryer",Iron,"translation missing: en.hosting_amenity_50"}'
 '{"Wireless Internet","Air conditioning",Kitchen,Heating,"Family/kid friendly",Washer,Dryer,"Smoke detector","Fire extinguisher",Essentials,Shampoo,Hangers,"Hair dryer",Iron,"translation missing: en.hosting_amenity_50"}'
 '{TV,"Cable TV","Wireless Internet","Air conditioning",Kitchen,Breakfast,"Buzzer/wireless intercom",Heating,"Family/kid friendly","Smoke detector","Carbon monoxide detector","Fire extinguisher",Essentials,Shampoo,Hangers,"Hair dryer",Iron,"Laptop friendly workspace","translation missing: en.hosting_amenity_50"}'
 ...
 '{Internet,"Wireless Internet","Air conditioning",Kitchen,Elevator,"Buzzer/wireless intercom",Heating,"Family/kid friendly",Washer,Dryer,"Smoke detector","Carbon monoxide detector","First aid kit","Fire extinguisher",Essentials,Shampoo,"Lock on bedroom door","24-hour check-in",Hangers,"Hair dryer","L

In [137]:
# Should be scaled as such:
# 0 - Real Bed
# 1 - Futon
# 2 - Pull-out Sofa
# 3 - Airbed
# 4 - Couch
print(df['bed_type'].unique())

['Real Bed' 'Futon' 'Pull-out Sofa' 'Airbed' 'Couch']


In [138]:
# Should be scaled as such:
# 0 - strict
# 1 - moderate
# 2 - flexible
# 3 - super_strict_30
# 4 - super_strict_60
print(df['cancellation_policy'].unique())

['strict' 'moderate' 'flexible' 'super_strict_30' 'super_strict_60']


In [139]:
# Can stay the same
print(df['cleaning_fee'].unique())

[ True False]


In [140]:
# Could perform sentiment analysis or could drop.
# I think dropping would be best
print(df['description'].unique())

['Beautiful, sunlit brownstone 1-bedroom in the loveliest neighborhood in Brooklyn. Blocks from the promenade and Brooklyn Bridge Park, with their stunning views of Manhattan, and from the great shopping and food.'
 "Enjoy travelling during your stay in Manhattan. My place is centrally located near Times Square and Central Park with easy access to main subways as well as walking distance to many popular restaurants and bus tours. My place is close to the subway, Totto Ramen, Hell's Kitchen, Ippudo Westside, Empanada Mama, Intrepid Sea, Air & Space Museum. My place has three true bedrooms and one bathroom. The kitchen is stocked with stainless steel appliances like the Keurig machine. The living room is spacious and can accommodate another person thanks to the pull out bed. My place is centrally located to some of the top attractions in the city. Feel free to explore the entire apartment and do not worry about sharing the space with any strangers. This is all yours during your stay. I a

In [141]:
# Should drop
print(df['first_review'].unique())

['2016-06-18' '2017-08-05' '2017-04-30' ... '2010-05-26' '2011-04-22'
 '2014-02-10']


In [142]:
# Should drop
print(df['host_has_profile_pic'].unique())

['t' nan 'f']


In [143]:
# Should replace with the following:
# True: t
# False: F or NaN
print(df['host_identity_verified'].unique())

['t' 'f' nan]


In [144]:
# Should convert to decimals, fill NaN with 0.
# Could convert to 'high' and 'low' if less than 50%
print(df['host_response_rate'].unique())

[nan '100%' '71%' '68%' '67%' '90%' '86%' '92%' '80%' '89%' '0%' '50%'
 '70%' '25%' '29%' '93%' '33%' '81%' '63%' '38%' '60%' '88%' '78%' '75%'
 '95%' '97%' '94%' '40%' '58%' '76%' '91%' '79%' '82%' '30%' '83%' '87%'
 '99%' '17%' '20%' '77%' '96%' '73%' '98%' '59%' '57%' '64%' '56%' '44%'
 '74%' '27%' '6%' '85%' '10%' '55%' '43%' '14%' '84%' '46%' '26%' '53%'
 '65%' '52%' '22%' '69%' '66%' '15%' '61%' '41%' '72%' '54%' '35%' '36%'
 '62%' '21%' '47%']


In [145]:
# Should convert to host_status the following:
# new - less than a year
# regular - 1 to 5 years
# verteran - longer than 5 years
print(df['host_since'].unique())

['2012-03-26' '2017-06-19' '2016-10-25' ... '2010-05-16' '2010-04-30'
 '2009-08-23']


In [146]:
# Should convert to:
# True - t
# False - f
print(df['instant_bookable'].unique())

['f' 't']


In [147]:
# Should use to determine distance to 4 landmarks using GeoPandas
# Empire State building
# Statue of Liberty
# Time Square
# Central Park
# JFK Airport
# Grand Central station
print(df['latitude'].unique())
print(df['longitude'].unique())

[40.69652363 40.76611542 40.80810999 ... 40.70902537 40.70674885
 40.73853473]
[-73.99161685 -73.98903992 -73.94375584 ... -73.93940479 -73.94237732
 -74.00015737]


In [148]:
# Drop
print(df['name'].unique())

['Beautiful brownstone 1-bedroom'
 'Superb 3BR Apt Located Near Times Square' 'The Garden Oasis' ...
 'one room bushwick' 'Modern 2 Bedroom Apartment in Williamsburg'
 "Designer's Apartment in HEART of NYC"]


In [149]:
# Drop
print(df['neighbourhood'].unique())

['Brooklyn Heights' "Hell's Kitchen" 'Harlem' 'Alphabet City'
 'Lower East Side' 'Flatbush' 'East Flatbush' 'Jamaica' 'Forest Hills'
 'Murray Hill' 'Lefferts Garden' 'Brownsville' 'Williamsburg'
 'East Village' 'Upper East Side' 'Bushwick' 'Bedford-Stuyvesant'
 'Chelsea' 'Greenwich Village' 'Washington Heights' 'Midtown East'
 'Park Slope' 'West Village' 'Astoria' 'East Harlem' 'Morningside Heights'
 'Tribeca' 'Financial District' 'Gramercy Park' 'Sunset Park'
 'Upper West Side' 'Greenpoint' 'Prospect Heights' 'Gravesend' 'Flushing'
 'Midtown' 'Soho' 'Roosevelt Island' 'Soundview' 'Crown Heights' 'Gowanus'
 'Boerum Hill' 'Greenwood Heights' 'Chinatown' 'Sunnyside'
 'Windsor Terrace' 'Sheepshead Bay' 'Clinton Hill' 'Westerleigh'
 'Ridgewood' 'Battery Park City' 'The Rockaways' 'Downtown Brooklyn'
 'Midwood' 'Fort Greene' 'Flatiron District' 'Bay Ridge' 'Inwood'
 'Kips Bay' 'Eastchester' 'Long Island City' 'Hamilton Heights'
 'Carroll Gardens' 'Rego Park' 'Bayside' 'Mott Haven' 'Jackson 

In [150]:
# Use in conjunction with review_scores_rating somehow
print(df['number_of_reviews'].unique())

[  2   6  10  82  26   5  57  14   4  40   1  17  11   0  34   3  31   9
  19  22  23   7 144  38 105  67  16  13  18  15  21  25 206  73  44  27
 167  43   8  28  81  12  63  30  42  75  60  52 135  64  36 191  24  66
  51  32 125 129  35  70  46  93  45  79  61  20  88  37 127 190  48  69
  59  72  54  53 117 101 106 255  47 114  29 192 113  91  49  62  84 146
 119  74  39  56  86  33 157  95  94 158  87 142  90 194  55 208 131 111
 133 102 148  89 147  85 139 202  71  78  41 159 156 100  65 207 172 136
  98 171  80  92 155 193 199 103  77 137 112  76 140  58 120  50 162 110
 107 246 195 273 251  83 351 152 116 189 134  96 141 109 118 121 175 104
  68 149 383 122 165 115 108 123 163 256 153 323 465 178  99 126 181 132
 220 128 154 218 150 182  97 225 198 224 201 288 138 343 306 169 236 173
 179 174 209 124 130 252 205 322 143 164 203 197 161 214 145 213 267 227
 196 268 269 168 184 242 232 177 151 200 211 264 228 238 217 241 298 326
 336 281 240 226 221 296 237 170 260 275 166 160 23

In [151]:
# Use in conjunction with review_scores_rating somehow
print(df['review_scores_rating'].unique())

[100.  93.  92.  86.  72.  89.  88.  96.  84.  nan  94.  80.  87.  95.
  85.  99.  70.  98.  97.  91.  90.  76.  60.  83.  82.  75.  55.  73.
  40.  20.  67.  71.  79.  78.  81.  47.  74.  68.  66.  50.  77.  65.
  53.  64.  69.  27.  62.  57.  56.  63.]


In [152]:
# Drop
print(df['thumbnail_url'].unique())

['https://a0.muscache.com/im/pictures/6d7cbbf7-c034-459c-bc82-6522c957627c.jpg?aki_policy=small'
 'https://a0.muscache.com/im/pictures/348a55fe-4b65-452a-b48a-bfecb3b58a66.jpg?aki_policy=small'
 'https://a0.muscache.com/im/pictures/6fae5362-9e3a-4fa9-aa54-bbd5ea26538d.jpg?aki_policy=small'
 ...
 'https://a0.muscache.com/im/pictures/55162426/6016bc16_original.jpg?aki_policy=small'
 'https://a0.muscache.com/im/pictures/7fbe448c-5293-4a22-a83e-54c8bc1bbf0d.jpg?aki_policy=small'
 'https://a0.muscache.com/im/pictures/b3971b63-06d9-4417-86ca-e6b40c22edca.jpg?aki_policy=small']


In [153]:
# Clean to only 5 digit numbers
print(df['zipcode'].unique())

['11201' '10019' '10027' '10009.0' '10002' '11226.0' '11212.0' '11411.0'
 '11374' '10016' '11225' '11212' '11211.0' '10003.0' '10021' '11237'
 '11233' '10001.0' '11249.0' '10037' '10011' '10011.0' '10032' '10022'
 '11226' '11217' '10031' '10014' '11101' '10029.0' '11221' nan '10025'
 '10013' '10040' '10039' '10038' '10003' '11219' '11222' '11238' '11206'
 '11223' '11355' '11220' '10036' '10024' '10012' '10044' '10023' '10028'
 '10473.0' '11249' '11238.0' '11232' '10010' '10002.0' '11377' '11216'
 '10128' '11207' '11213' '11215' '11229' '11216.0' '10314' '10075' '11385'
 '10282' '10035.0' '11218' '11378' '11692' '11230' '10065' '11205' '11209'
 '11237.0' '11225.0' '11213.0' '11206.0' '11105' '10034' '10030' '10005'
 '10475' '11231.0' '11103' '10017' '11102' '11361' '10455' '11428' '10033'
 '11370.0' '11234.0' '11211' '10305' '11354' '11205.0' '11204' '11372'
 '11207.0' '10314.0' '10466' '11417' '10004' '10026' '11236.0' '10035'
 '11201.0' '11412' '10006' '11234' '10468' '10018' '11203.0

In [154]:
# Drop NaN
print(df['bedrooms'].unique())

[ 1.  3.  2.  0.  4.  5.  7. nan  6. 10.  8.  9.]


In [155]:
# Drop NaN's
print(df['beds'].unique())

[ 1.  3.  2.  6.  4.  5. nan  7.  8. 10. 13.  9. 12. 16. 11.  0. 15. 18.]


In [156]:
################# Cleaning ###########################

In [157]:
# Take only neccessary columns
df = df[['log_price', 'property_type', 'room_type', 'amenities',
       'accommodates', 'bathrooms', 'bed_type', 'cancellation_policy',
       'cleaning_fee', 'host_identity_verified', 'host_response_rate',
       'host_since', 'instant_bookable', 'latitude',
       'longitude', 'number_of_reviews', 'review_scores_rating',
        'zipcode', 'bedrooms', 'beds']]

In [158]:
# FUNCTION one_hot_encode
# one_hot_encodes the dataframe for a given column
def one_hot_encode(dataframe, column, pre_name = ""):
    unique_vals = dataframe[column].unique()
    
    # Create a one_hot_encoding for each unique value in the column
    for val in unique_vals:
        orig_num_values = len(dataframe[dataframe[column] == val])
        
        column_name = pre_name + val
        dataframe[column_name] = dataframe[column].apply(lambda value: 1 if value == val else 0)
        
        new_num_values = len(dataframe[dataframe[column_name] == 1])
        
        # Ensures the new number of values matches the old, such that no values are errased in the process
        assert(orig_num_values == new_num_values)
    return dataframe

In [159]:
# FUNCTION create_amenities_list
# Only needs to be run to find all unique amenities
def create_amenities_list(dataframe, column = 'amenities'):
    all_amenities = []
    
    for i, amenities_str in dataframe[column].items():
        amenities_list = [item.strip().strip('"') for item in amenities_str.strip('{}').split(',')]
        amenities_list = amenities_list[:-1]
        
        # Append to all_amenities to build the set of all unique amenities
        all_amenities.extend(amenities_list)
        
        # Replace the original string in the DataFrame with the parsed list
        dataframe.at[i, column] = amenities_list

    # Return unique amenities if needed
    return list(set(all_amenities))

# Call the function
# amen = create_amenities_list(df)
#print(amen)

In [160]:
# Desired unique amenities:
desired_unique_amenities = ['Waterfront', 'Washer / Dryer', 'Pets allowed', 'Wireless Internet', 
 'Air conditioning', 'Family/kid friendly', 'Dishwasher', 'Kitchen', 
 'Beachfront', 'Heating', 'TV', 'Pool']

# FUNCTION amenities_one_hot
# Performs one hot on the amenities column
def amenities_one_hot(dataframe, desired_amenities, column='amenities'):
    # Initialize new columns for each desired amenity with a default value of 0
    for amenity in desired_amenities:
        dataframe[amenity] = 0

    # Use apply to set the value of each amenity column to 1 if it's in the list of amenities for that row
    def set_amenity_flags(amenities_list):
        for amenity in amenities_list:
            if amenity in desired_amenities:
                dataframe.loc[dataframe.index, amenity] = 1
    
    dataframe[column].apply(set_amenity_flags)
    
    return dataframe

df = amenities_one_hot(df, desired_unique_amenities)

print(df.columns)

Index(['log_price', 'property_type', 'room_type', 'amenities', 'accommodates',
       'bathrooms', 'bed_type', 'cancellation_policy', 'cleaning_fee',
       'host_identity_verified', 'host_response_rate', 'host_since',
       'instant_bookable', 'latitude', 'longitude', 'number_of_reviews',
       'review_scores_rating', 'zipcode', 'bedrooms', 'beds', 'Waterfront',
       'Washer / Dryer', 'Pets allowed', 'Wireless Internet',
       'Air conditioning', 'Family/kid friendly', 'Dishwasher', 'Kitchen',
       'Beachfront', 'Heating', 'TV', 'Pool'],
      dtype='object')


In [161]:
# FUNCTION boolean_encode
# Encodes boolean with True = 1, False = 0
def boolean_encode(dataframe, column, t = 't'):
    # Drop NaN values
    dataframe = dataframe.dropna(subset=[column])
    
    dataframe[column] = dataframe[column].apply(lambda value: 1 if value == True or value == t else 0)
    
    return dataframe

In [162]:
# FUNCTION fix_host_response_rate
# Fixes the string percentages into doubles
def fix_host_response_rate(dataframe, column = 'host_response_rate'):
    dataframe = dataframe.dropna(subset=[column])
    
    dataframe[column] = dataframe[column].str.replace('%', '', regex=False).astype(float) / 100
    
    return dataframe

In [163]:
# FUNCTION find_num_years_hosting
# Finds the number of years a host has been hosting
def find_num_years_hosting(dataframe, column = 'host_since'):
    dataframe[column] = pd.to_datetime(dataframe[column])
    current_year = datetime.now().year
    
    dataframe['num_years_hosting'] = current_year - dataframe[column].dt.year
    
    dataframe.drop(columns=[column], inplace=True)

    return dataframe

In [164]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of the Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c  # Distance in kilometers
    return distance

# FUNCTION find_distance_to_landmarks
# Finds the distance from an air_bnb to each landmark
def find_distance_to_landmarks(dataframe, landmarks, lat_column='latitude', lon_column='longitude'):
    # Iterate through landmarks
    for landmark in landmarks:
        column_name = "distance_to_" + landmark[0]
        
        # Calculate distance to each landmark for each row in the DataFrame
        distances = []
        for index, row in dataframe.iterrows():
            distance = haversine(row[lat_column], row[lon_column], landmark[1], landmark[2])
            distances.append(distance)
        
        # Assign distances to the new column in the DataFrame
        dataframe[column_name] = distances

    return dataframe

In [165]:
# FUNCTION fix_zip_codes
# Function to remove any rows where the 'zipcode' column has more 
# than five chars
def fix_zip_codes(dataframe, column='zipcode'):
    dataframe = dataframe.dropna(subset=[column])
    dataframe = dataframe[dataframe[column].str.len() <= 5]
    
    return dataframe

In [166]:
#####################################################
# EXECUTION
#####################################################

In [167]:
######################################################
# ONE HOT ENCODINGS
######################################################
# One-hot-encode 'property_type'
df = one_hot_encode(df, 'property_type')

# One-hot-encode 'room_type'
df = one_hot_encode(df, 'room_type')

# One-hot-encode bed_type
df = one_hot_encode(df, 'bed_type')

#One-hot-encode cancellation_policy
df = one_hot_encode(df, 'cancellation_policy', pre_name = 'cancellation_policy_')
######################################################

In [168]:
######################################################
# BOOLEAN ENCODINGS
######################################################
# One-hot-encode 'cleaning_fee'
df = boolean_encode(df, 'cleaning_fee')

# One-hot-encode 'host_identity_verified'
df = boolean_encode(df, 'host_identity_verified')

df = boolean_encode(df, 'instant_bookable')
######################################################

C:\Users\roman\AppData\Local\Temp\ipykernel_22484\3017023220.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column] = dataframe[column].apply(lambda value: 1 if value == True or value == t else 0)


In [169]:
######################################################
# fix host_response rate
######################################################
df = fix_host_response_rate(df)
######################################################

C:\Users\roman\AppData\Local\Temp\ipykernel_22484\4204321114.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column] = dataframe[column].str.replace('%', '', regex=False).astype(float) / 100


In [170]:
######################################################
# Find number of years a host has been hosting
######################################################
df = find_num_years_hosting(df)
######################################################

In [171]:
landmarks = [('Times_Square', 40.758896, -73.985130), 
             ('Empire_State_Building', 40.7484, -73.9857),
             ('Statue_of_Liberty', 40.6892, -74.0445),
             ('Central_Park',40.7826, -73.9656),
             ('JFK_Airport', 40.6446, -73.7797)]

######################################################
# Find the distance of each airBnB to every landmark in
# kilometers
######################################################
df = find_distance_to_landmarks(df, landmarks)
######################################################

In [172]:
df = fix_zip_codes(df)

######################################################
# Drop all rows where 'zipcode' has more than five chars
######################################################
df = fix_zip_codes(df)
######################################################

In [182]:
# Drop rows with NaN's in bedrooms, beds, accommodates, or log_price columns
df = df.dropna(subset=['bedrooms', 'beds', 'accommodates', 'log_price'])

In [183]:
print(len(df))

16270


In [184]:
print(df.columns)

Index(['log_price', 'property_type', 'room_type', 'amenities', 'accommodates',
       'bathrooms', 'bed_type', 'cancellation_policy', 'cleaning_fee',
       'host_identity_verified', 'host_response_rate', 'instant_bookable',
       'latitude', 'longitude', 'number_of_reviews', 'review_scores_rating',
       'zipcode', 'bedrooms', 'beds', 'Waterfront', 'Washer / Dryer',
       'Pets allowed', 'Wireless Internet', 'Air conditioning',
       'Family/kid friendly', 'Dishwasher', 'Kitchen', 'Beachfront', 'Heating',
       'TV', 'Pool', 'Apartment', 'House', 'Loft', 'Hostel', 'Condominium',
       'Townhouse', 'Other', 'Bed & Breakfast', 'Timeshare',
       'Serviced apartment', 'Guesthouse', 'Bungalow', 'Boutique hotel',
       'Guest suite', 'Boat', 'Villa', 'Dorm', 'Vacation home', 'In-law',
       'Chalet', 'Yurt', 'Cabin', 'Tent', 'Earth House', 'Castle',
       'Entire home/apt', 'Private room', 'Shared room', 'Real Bed', 'Futon',
       'Pull-out Sofa', 'Airbed', 'Couch', 'cancellatio

In [186]:
df.to_csv('data_for_KNN.csv', index = False)